# Financial App

In [ ]:
import os
from pathlib import Path
import pandas as pd

# Get the current working directory
root_path = Path(os.getcwd())
if  root_path.name == "notebooks":
    print("✅ La última carpeta es 'notebooks'.")
    os.chdir(f"{root_path}/..")
    print(f"Now in directory: {os.getcwd()}")
else:
    print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
import app.modules.database.structur_databases as sdb
import app.modules.database.companies as companies
import app.modules.database.customers as customers
import app.modules.database.credit_manager as credit_manager
import app.modules.database.portfolio_manager as portfolio_manager
import app.modules.database.collection as colltions

from app.modules.database.credit_manager import credits_balance

from importlib import reload

reload(customers)
reload(companies)
reload(portfolio_manager)
reload(credit_manager)

%run logs/run_log.py

✅ La última carpeta es 'notebooks'.
Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
✅ The system correctly imports all modules.
✅ SQL script executed successfully.
✅ 24 provinces successfully inserted into the database.
✅ The company NeoCrediT S.A. has been added.
✅ New business plan created, its ID is 1 and it was associated with the company NeoCrediT S.A..
✅ The company Onoyen S.R.L. has been added.
✅ New business plan created, its ID is 2 and it was associated with the company Onoyen S.R.L..
✅ The company Asociación Mutual Union Federal has been added.
✅ New business plan created, its ID is 3 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 4 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 5 and it was associated with the company Asociación Mutual Union Federal.
✅ Settings successfully updated in the database.
✅ Supplier 2 

In [ ]:
pp = pd.read_sql('portfolio_purchases', engine, index_col='ID')
customers = pd.read_sql('customers', engine, index_col='ID')
credits = pd.read_sql('credits', engine, index_col='ID')
collections = pd.read_sql('collection', engine, index_col='ID')
installments = pd.read_sql('installments', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')
bp = pd.read_sql('business_plan', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')

balance = credits_balance().copy()

print(f'Total Onoyen S.R.L: $ {balance.loc[balance['Anchorer'] == 'Onoyen S.R.L.', 'Total'].sum():,.2f}')
print(f'Total AMUF: $ {balance.loc[balance['Anchorer'] != 'Onoyen S.R.L.', 'Total'].sum():,.2f}')


balance = balance.loc[balance['Anchorer'] != 'Onoyen S.R.L.']

print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}\n')

balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)

balance

''' $ 69.928.978,01 	 $ 82.781.943,50 	 $ 17.384.208,05 	 $ 170.095.129,56 	 $ 91.668.437,88 '''

Total Onoyen S.R.L: $ 0.00
Total AMUF: $ 91,668,436.69
Capital: $ 43,723,832.30
Interés: $ 39,623,639.83
IVA: $ 8,320,964.33
Total: $ 91,668,436.69



' $ 69.928.978,01 \t $ 82.781.943,50 \t $ 17.384.208,05 \t $ 170.095.129,56 \t $ 91.668.437,88 '

In [ ]:
# ✅ Load and process the credit report
df = pd.read_excel(f"inputs/Reporte - Inv. Créditos.xlsx")
print(f'Last issue of a credit: {df['F. Liq.'].max()}')

# ✅ Clean and structure data
df["ID"] = df["Clave Externa"].str.lstrip("CH_")
df.set_index("ID", inplace=True)
df.dropna(axis=1, how="all", inplace=True)
df = df.query("Fondeador == 'NEOCREDIT'")
inv = df.copy()

Last issue of a credit: 2025-03-18 00:00:00


In [ ]:
coll_path = 'inputs/Reporte - Cobranzas.xlsx'

df = pd.read_excel(coll_path)
df.rename(columns={'Emisión': 'D_Emission', 'Tipo Cobranza': 'Type_Collection', 'Crédito': 'ID_Op', 'Cta.': 'ID_Inst', 'CA': 'Capital', 'IN': 'Interest', 'IV': 'IVA', 'TOTAL': 'Total'}, inplace=True)
df.drop(columns=['GS', 'PU'], inplace=True)
df = df.loc[df['Línea'] != 'DECRETO 14/12']
df = df.loc[(df['Type_Collection'] != 'ANTICIPO')]
df['Per. Ems.'] = df['D_Emission'].dt.to_period('M')

amuf = list(df['Línea'].unique())
amuf.remove('PENALTY')
df['Línea'] = df['Línea'].replace(amuf, 'Asociación Mutual Union Federal')

df['ID_Ext'] = df['ID_Op'].apply(lambda x: inv.loc[inv['Id. Op.'] == x].index[0] if x in inv['Id. Op.'].values else None)

In [ ]:
reload(colltions)
coll = df.loc[df['Línea'] != 'PENALTY'].groupby(['D_Emission', 'Línea', 'Type_Collection', 'ID_Ext'])[['Total']].sum()
type_coll = colltions.TypeDataCollection

for d, l, t, c in coll.index:
    filtro = (df['D_Emission'] == d) & (df['Línea'] == l) & (df['ID_Op'] == c)
    if 'COBRANZA X CANCEL ANT' in df.loc[filtro, 'Type_Collection'].values:
        amount = -coll.loc[(d,l,t,'COBRANZA'), 'Total']
        print(f'Cobrando el cr. nro. {c} el {d} por ${amount:,.2f}')
        colltions.collection_w_early_cancel(type_coll.ID_Ext, c, amount, 3, d,True)
    else:
        amount = -coll.loc[(d,l,t, c), 'Total']
        print(f'Cobrando el cr. nro. {c} el {d} por ${amount:,.2f}')
        colltions.charging(type_coll.ID_Ext, c, amount, 3, d, True)

Cobrando el cr. nro. 76674 el 2023-11-10 00:00:00 por $17,438.51
   Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
0  2210.82  12584.87  2642.82  17438.51     2731 2023-11-10           COMUN
Cobrando el cr. nro. 76718 el 2023-11-10 00:00:00 por $11,625.68
   Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
0  1473.88   8389.91  1761.88  11625.68     2779 2023-11-10           COMUN
Cobrando el cr. nro. 76734 el 2023-11-10 00:00:00 por $20,344.93
   Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
0  2579.29  14682.35  3083.29  20344.93     2755 2023-11-10           COMUN
Cobrando el cr. nro. 76740 el 2023-11-10 00:00:00 por $21,798.14
   Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
0  2763.53  15731.09  3303.53  21798.14     2743 2023-11-10           COMUN
Cobrando el cr. nro. 76756 el 2023-11-10 00:00:00 por $23,251.35
   Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Co

In [ ]:
penalty = -df.loc[df['Línea'] == 'PENALTY'].groupby(['D_Emission', 'CUIL'])[['Total']].sum()

for d, cuil in penalty.index:
    amount = penalty.loc[(d,cuil), 'Total']
    print(f'Penalty a {cuil} el {d} por ${amount:,.2f}')
    colltions._process_penalty(amount, credits, d, )

Penalty a 20363206566 el 2024-06-05 00:00:00 por $16,208.73
    Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
24      0.0  13395.64  2813.09  16208.73     5231 2024-06-05      ANTICIPADA
Penalty a 27144916773 el 2024-06-05 00:00:00 por $24,745.21
    Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
12      0.0  20450.59  4294.62  24745.21     5844 2024-06-05         PENALTY
Penalty a 27298789367 el 2024-06-05 00:00:00 por $36,173.98
    Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
12      0.0  29895.85  6278.13  36173.98     5845 2024-06-05         PENALTY
Penalty a 23101574849 el 2024-08-02 00:00:00 por $11,327.68
    Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
12      0.0   9361.72  1965.96  11327.68     5846 2024-08-02         PENALTY
Penalty a 27228688288 el 2024-08-02 00:00:00 por $21,246.12
    Capital  Interest      IVA     Total  ID_Inst D_Emission Type_Collection
24     